# Vision Module Diagnosis: The "Black and White" Problem
 
## 1. The Problem
 You observed that the vision module output was patchy, noisy, or completely missing parts of the object.
 The image looked like a "broken" black and white mask.

## 2. Why this happens
 The issue is **Global Thresholding** vs. **Uneven Lighting**.
 
 * **How Computers See:** Computers see images as a grid of numbers (0=Black, 255=White).
 * **The Old Method (Global):** It picks ONE number (e.g., 127). "If pixel > 127, make it white. Else, black."
 * **The Reality:** In real life, shadows exist. A white object in a shadow might have a brightness of 100. A dark background in bright sunlight might have a brightness of 150.
 * **The Failure:** The Global Threshold separates them wrong because the *light* changed, not the *object*.

## 3. The Solution
We use **Adaptive Thresholding**. Instead of one global number, the algorithm calculates a threshold for *every single pixel* based on its neighbors. It asks: "Is this pixel brighter than the pixels immediately around it?"


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from vision import VisionProcessor
def show_comparison(title1, img1, title2, img2):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img1, cmap='gray' if len(img1.shape)==2 else None)
    plt.title(title1)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(img2, cmap='gray' if len(img2.shape)==2 else None)
    plt.title(title2)
    plt.axis('off')
    plt.show()

## Step 1: Reproduce the Failure
Here we simulate what your old code was likely doing (Global Thresholding).

In [ ]:
# Load the image (Replace with your actual filename)
img_path = 'image_4db42a.png' 

# Initialize our processor
processor = VisionProcessor()
try:
    original_img = processor.load_image(img_path)
except Exception as e:
    # Create a dummy image if file not found for demonstration
    print(f"Warning: {e}. Creating dummy image.")
    original_img = np.zeros((300, 300, 3), dtype=np.uint8)
    cv2.circle(original_img, (150, 150), 100, (255, 255, 255), -1)
    # Add fake shadow
    for i in range(300):
        original_img[:, i] = original_img[:, i] * (i / 300)

# Simulate the BROKEN method (Global Threshold)
gray = cv2.cvtColor(original_img, cv2.COLOR_RGB2GRAY)
_, bad_mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

show_comparison("Original Image (Note Lighting/Shadows)", original_img, 
                "FAILED: Global Thresholding", bad_mask)

**Observation:** Notice how parts of the object disappear if they are in the shadow? Or background noise appears if it's too bright? This matches the failure you described.

## Step 2: Apply the Fix (Adaptive Thresholding)
Now we use the function from `vision.py`. It looks at local contrast rather than global brightness.


In [ ]:
#Run the robust method
# block_size=11 checks an 11x11 square around every pixel
robust_mask = processor.get_binary_mask_robust(original_img, block_size=11, c_value=2)

show_comparison("FAILED: Global Thresholding", bad_mask, 
                "FIXED: Adaptive Thresholding", robust_mask)

## Conclusion
1.  **Global Thresholding** fails when lighting is not perfect.
2.  **Adaptive Thresholding** (used in `vision.py`) succeeds because it adapts to shadows.

**Next Steps:**
 Use `vision.py` in your main codebase. If you still see noise (small dots), try increasing the `block_size` in `get_binary_mask_robust` (must be an odd number, e.g., 15, 21).

# References for Image Thresholding & Lighting Correction

Source: OpenCV Official Documentation - Image Thresholding

Source: Digital Image Processing by Rafael C. Gonzalez and Richard E. Woods. (Chapter 10)